Misc derivations of frame3d FEM elements and vefirication that results are consistent with welib.FEM.frame3d

In [ ]:
# Sympy
import sympy as sp
from sympy import symbols, Symbol, simplify, Function, expand_trig
from sympy import cos,sin
from sympy import diff, integrate
from sympy import python, latex
from sympy import Matrix
from sympy.physics.vector import init_vprinting, vlatex
init_vprinting(use_latex='mathjax', pretty_print=False)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import matplotlib.pyplot as plt
import numpy as np
%load_ext autoreload
%autoreload 2
%matplotlib inline
# --- local packages
from welib.FEM.frame3d import *
from welib.FEM.fem_sympy import *

In [ ]:
s = Symbol('s')
x = Symbol('x')
L = Symbol('L')
EIx,EIy,EIz,GKt,EA=symbols('EIx,EIy,EIz,GKt,EA') 
Ix,Iy,Iz,Kt=symbols('Ix,Iy,Iz,Kt') 
rho = Symbol('rho')
E, G = symbols('E, G')
A = Symbol('A')
m = Symbol('m')
D,t = sp.symbols('D, t')
T=Symbol('T')   # Axial load
ux1,uy1,uz1,tx1,ty1,tz1,ux2,uy2,uz2,tx2,ty2,tz2 = symbols('u_x1, u_y1, u_z1, theta_x1, theta_y1, theta_z1, u_x2, u_y2, u_z2, theta_x2, theta_y2, theta_z2')
n1,n2,n3,n4,n5,n6=symbols('n_1,n_2,n_3,n_4,n_5,n_6')
q=Matrix([[ux1,uy1,uz1,tx1,ty1,tz1,ux2,uy2,uz2,tx2,ty2,tz2]]).T

In [ ]:
# --- Look at properties
#print('h in polynomial form')
#h(x, u2, u3,u5,u6,L).simplify().expand().collect(x)
#print('u in polynomial form')
#uax(x, u1, u4, L).simplify().expand().collect(x)


In [ ]:
# --- Interpolation matrix from nodal DOF to deflections
# Symbolic version as function of shape functions
NN1 = zeros(4,12)
NN1[0,0]=n1
NN1[1,1]=n2
NN1[2,2]=n2
NN1[3,3]=n1
NN1[2,4]=-n3
NN1[1,5]=n3
NN1[0,6]=n4
NN1[1,7]=n5
NN1[2,8]=n5
NN1[3,9]=n4
NN1[2,10]=-n6
NN1[1,11]=n6

# Explicit version
NN2 = zeros(4,12)
NN2[0,0]=b1(x/L)
NN2[1,1]=b2(x/L)
NN2[2,2]=b2(x/L)
NN2[3,3]=b1(x/L)
NN2[2,4]=-b3(x/L,L)
NN2[1,5]=b3(x/L,L)
NN2[0,6]=b4(x/L)
NN2[1,7]=b5(x/L)
NN2[2,8]=b5(x/L)
NN2[3,9]=b4(x/L)
NN2[2,10]=-b6(x/L,L)
NN2[1,11]=b6(x/L,L)

# Explicit version using frame3d module
NN3 = frame3d_N(x,L)
NN1
NN3

In [ ]:
# --- Mass matrix
from welib.FEM.fem_sympy import *
rx = symbols('r_x')
Me= massMatrixFromShapeFunctions(NN2, x, L, m, coeffs=[m/L,m/L,m/L,m/L*rx**2])
Ke1, Me1, _= frame3d_KeMe(E,G,Kt,EA,EIx,EIy,EIz,L,A,m) 

Me1=Matrix(Me1).subs(EIx/A/E,rx**2)
#Me1-Me
#Me/m*2*105
Me1/m*2*105
Me/m*2*105
#Me-Me1
#Me
#print(python(M))


In [ ]:
from welib.FEM.fem_sympy import *
# --- Define Stiffness matrix based on Strain energy
# Displacements
u = frame3d_u(x,q,L)
u_x=u[0]
u_y=u[1]
u_z=u[2]
t_x=u[3]

# Regular strain energy - TODO cross terms migth be missing
UBy = frame3d_U_B(u_y,x,L,EIz)  # strain energy bending along y (EIz)
UBz = frame3d_U_B(u_z,x,L,EIy)  # strain energy bending along z (EIy)
UA = frame3d_U_A(u_x,x,L,EA)    # axial energy
UT = frame3d_U_A(t_x,x,L,GKt)    # axial energy
# Energy from axial loads - OK
UGy = frame3d_U_G(u_y,x,L,T)
UGz = frame3d_U_G(u_z,x,L,T)
#            
Ke= stiffnessMatrixFromPot(UA+UBy+UBz+UT, q)
Kg= stiffnessMatrixFromPot(UGy+UGz, q)
Ke1, Me1, _= frame3d_KeMe(E,G,Kt,EA,EIx,EIy,EIz,L,A,m) 
Ke1=Matrix(Ke1).subs([(G*Kt,GKt)])
#            
Ke
Kg

#print(python(Kg))
#print(python(Ke))

In [ ]:
# --- Numerical examples
Subst=[(L,100),(D,8),(t,0.045), (rho,7850), (E,210e9),(G,79.3e9)]
A  = np.pi*( (D/2)**2 - (D/2-t)**2)
Ix = np.pi/32*(D**4-(D-2*t)**4)  # Polar second moment of area [m^4]

Mass = rho*A*L

Me_num=np.array(Me.subs([(m,Mass),(rx,np.sqrt(Ix/A))]+Subst)).astype(np.float)
Me_num[3,9]

In [ ]:
# --- latex export
print(cleantex(latex(Me/m*2*105)))
print(cleantex(latex(Matrix(Ke1))))
print(cleantex(latex(Matrix(Kg)/T)))
#print(cleantex(latex(Matrix(I))))

In [ ]:
np.array(Kg/T)